# RC4 and WEP

* WEP uses RC4 to encrypt packets for transmission over IEEE 802.11 (WLAN)
* Every packet must be encrypted with a different RC4 key  to prevent calculation of the keystream
* RC4 key = 24-bit IV + 40 or 104-bit long term key
* 24-bit IV is short (2^24 ~ 16 million different short term keys)

## RC4

RC4 (Rivest Cipher 4) is a stream cipher from Ron Rivest (1987). It consists of two phases.

### Key-Scheduling-Algorithm (KSA)

The Key-Scheduling-Algorithm (KSA) produces an array-state vector S of length N. S is a pseudo-random permutation S of the values [0..N-1]. It depends on the input key K.

For `n = 8`, `N = 2^8 = 256`. This will result in `2^8! ~ 2^1700` possible permutations for the internal state S. It's big enough to store keys of variable sizes.

In [ ]:
from typing import List, Tuple

def swap(input_list: List[int], i: int, j: int) -> List[int]:
    """
    Swaps two elements at position i and j in a list in O(1)
    
    Keyword arguments:
    input_list -- list of numbers
    i -- 0-based index in list
    j -- 0-based index in list
    """
    # store in tuple
    temp_tuple: Tuple[int, int] = input_list[i], input_list[j]
    # unpack again
    input_list[j], input_list[i] = temp_tuple
      
    return list

def KSA(key: List[int], N: int) -> List[int]:
    """
    Returns a pseudo-random permutation of the numbers
    0 to N-1 depending on the key k in N rounds
    
    Keyword arguments:
    key -- key K, must be multiple of word length n
    N -- number of rounds = length of state-vector S
    """
    # initialization phase
    S: List[int] = []        #print(f'i = {i}; S[] = {S}\n')
    for i in range(N):
        S.append(i)
    j: int = 0
    #print('after initialization phase:')
    #print(f'S[] = {S}\n')
        
    # scrambling phase
    #print('scrambling phase')
    for i in range(N):
        # generate pseudo-random value for j
        j = (j + S[i] + key[i % len(key)]) % N
        swap(S, i, j)
        #print(f'i = {i}; S[] = {S}\n')
        
    return S
        
n = 8
N = 2**n # number of rounds
key = [35, 54, 155, 213, 167, 47, 16, 31]
S: List[int] = KSA(key, N)

print(f'S[] = {S}')

### Pseudo Random Generation Algorithm (PRGA)

The Pseudo Random Generation Algorithm (PRGA) uses the permutated state-vector S to generate a stream cipher. S gets continuously permutated. After each permutation one byte of the cipher stream is returned.

In [ ]:
def PRGA(S: List[int], length: int) -> List[int]:
    i: int = 0
    j: int = 0
    key_stream: List[int] = []
    
    N: int = len(S)
    while(True):
        i = (i + 1) % N
        print(f'i = {i}')
        j = (j + S[i]) % N
        print(f'j = {j}')
        print(f'before swap: S[i] = {S[i]}, S[j] = {S[j]}')
        swap(S, i, j)
        print(f'after swap: S[i] = {S[i]}, S[j] = {S[j]}')
        index: int = (S[i] + S[j]) % N
        print(f'index: {index}')
        print(f'S[{index}] = {S[index]}')
        key_stream.append(S[index])
        if i == length:
            break
    
    return key_stream

key_stream: str = PRGA(S, 15)
print(f'stream cipher s = : {key_stream}') 

### Attacks

- [What's wrong with WEP (2001)](https://www.researchgate.net/profile/Stefaan-Seys/publication/265230247_The_Insecurity_of_IEEE_80211_or_What%27s_Wrong_With_WEP/links/56250fb208aeabddac91b5c3/The-Insecurity-of-IEEE-80211-or-Whats-Wrong-With-WEP.pdf)
- [Weaknesses in the Key Scheduling Algorithm of RC4 (2001)](https://link.springer.com/chapter/10.1007/3-540-45537-X_1)
- [Breaking 104 Bit WEP in Less Than 60 Seconds (2007)](https://link.springer.com/chapter/10.1007/978-3-540-77535-5_14)


The security of RC4 only relies on the secret key `k` and the initialization vector `IV`. For the same (`k`, `IV`) RC4 is deterministic.

```
s = RC4(k, IV)
```

The reuse of the stream cipher `s` makes RC4 vulnerable to COA and KPA.